In [1]:
import pandas as pd

# データの読み込み
reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')

# 第３章　集約
## 3-1 データ数，種類数の算出
最も基本的な集約処理として，データ数のカウントがあります．これは，対象となるデータのレコード数（行数）をカウントする処理です．この他にもよく利用するカウント処理として，データのユニークカウントがあります．ユニークカウントとは，対象となるデータから同じ値のレコードを排除した後にレコード数をカウントする処理です．つまり，データの値の種類をカウントします．

### Q:カウントとユニークカウント
対象のデータセットは，ホテルの予約レコードです．\
予約テーブルから，ホテルごとに予約件数と予約したことがある顧客数を算出しましょう．\
\
ヒント：agg関数で集約処理をまとめて指定するやりやすいです

In [8]:
# agg関数を利用して、集約処理をまとめて指定
# reserve_idを対象にcount関数を適用
# customer_idを対象にnunique関数を適用
result_1 = reserve_tb \
  .groupby('hotel_id') \
  .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

# reset_index関数によって、列番号を振り直す（inplace=Trueなので、直接resultを更新）
result_1.reset_index(inplace=True)
result_1.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']


## 3-2　合計値の算出
### Q:合計値
対象のデータセットはホテルの予約レコードです．\
予約テーブルから，ホテルごとの宿泊人数別の合計予約金額を算出しましょう．

In [4]:
# 集約単位をhotel_idとpeople_numの組み合わせを指定
# 集約したデータからtotal_priceを取り出し、sum関数に適用することで売上合計金額を算出
result_2 = reserve_tb \
  .groupby(['hotel_id', 'people_num'])['total_price'] \
  .sum().reset_index()

# 売上合計金額の列名がtotal_priceになっているので、price_sumに変更
result_2.rename(columns={'total_price': 'price_sum'}, inplace=True)

## 3-3 極値，代表値の算出
### Q:代表値
対象のデータセットはホテルの予約レコードです．
予約テーブルから，ホテルごとの予約金額の最大値，最小値，平均値，中央値，20パーセンタイル値を算出しましょう．

In [9]:
# total_priceを対象にmax/min/mean/median関数を適用
# Pythonのラムダ式をagg関数の集約処理に指定
# ラムダ式にはnumpy.percentileを指定しパーセントタイル値を算出（パーセントは20指定）
import numpy as np

result_3 = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['max', 'min', 'mean', 'median',
                        lambda x: np.percentile(x, q=20)]}) \
  .reset_index()
result_3.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean',
                  'price_median', 'price_20per']

## 3-4 ばらつき具合の算出
### Q:分散値と標準偏差
対象のデータセットはホテルの予約レコードです．\
予約テーブルから，各ホテルの予約金額の分散値と標準偏差値を算出しましょう．\
ただし，予約が１件しかない場合は，分散値と標準偏差値を０としましょう．

In [10]:
# total_priceに対して、var関数とstd関数を適用し、分散値と標準偏差値を算出
result_4 = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['var', 'std']}).reset_index()
result_4.columns = ['hotel_id', 'price_var', 'price_std']

# データ数が1件だったときは、分散値と標準偏差値がnaになっているので、0に置き換え
result_4.fillna(0, inplace=True)


## 3-5 最頻値の算出
最頻値は最も多く出現しているあたいのことです．\
数値でも，カテゴリ値に変換することによって最頻値を利用できます．\
例えば，数値を四捨五入で整数かしたり，100ごとのレンジで値をカテゴリ化して，最頻値を利用します．
### Q:最頻値
対象のデータセットは，ホテルの予約レコードです．予約テーブルの予約金額を1000単位にカテゴリ化して最頻値を算出しましょう．

In [11]:
# round関数で四捨五入した後に、mode関数で最頻値を算出
reserve_tb['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

## 3-6 順位の算出
対象データを絞る際に順位を利用したり，複雑な時系列の結合をする際に時間順に順位づけし，結合条件に利用することもできます．\
順位づけする際には，計算こづとに注意する必要があります．順位づけには並び替えを実地する必要があり，データ数が多いと計算コストが跳ね上がってしまうからです．\
順位付けをする計算は，Window関数を利用すると柑橘かつ計算パフォーマンスよくかけます．Window関数は集約関数の１つですが，通常の集約関数とは違う点があります．それは，行を集約せず，集約した値を計算してから各行に付与するという点です．
### Q:時系列に番号を付与
対象のデータセットは，ホテルの予約レコードです．\
予約テーブルを利用して，顧客ごとに予約日時の順位を古い順につけましょう．\
同じ予約日時の場合は，データ行の読み込み順に小さな順位をつけましょう．


In [12]:
# rank関数で並び替えるために、データ型を文字列からtimestamp型に変換
# （「第10章 日時型」で解説）
reserve_tb['reserve_datetime'] = pd.to_datetime(
  reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S'
)

# log_noを新たな列として追加
# 集約単位の指定はgroup_byを利用
# 顧客ごとにまとめたreserve_datetimeを生成し、rank関数によって順位を生成
# ascendingをTrueにすることで昇順に設定(Falseだと降順に設定)
reserve_tb['log_no'] = reserve_tb \
  .groupby('customer_id')['reserve_datetime'] \
  .rank(ascending=True, method='first')

### Q:ランキング
対象のデータセットは，ホテルの予約レコードです．\
予約テーブルを利用して，ホテルごとの予約数に順位付けしましょう．\
同じ予約数の場合は，同予約数の全ホテルに最小の順位をつけましょう

In [13]:
# 予約回数を計算(「3-1 データ数、種類数の算出」の例題を参照)
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

# 予約回数をもとに順位を計算
# ascendingをFalseにすることで降順に指定
# methodをminに指定し、同じ値の場合は取り得る最小順位に指定
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'] \
  .rank(ascending=False, method='min')

# 必要のないrsv_cntの列を削除
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)